# Filtering and Restoring of Mixed and Noise Polluted Audio Samples Using Different Approaches

In [ ]:
import os
import soundfile as sf
import numpy as np
import matplotlib.pyplot as plt

inp_folder = 'audio'
outp_folder = 'output'

In [ ]:
file_names = ['speech.wav', 'street-noise.wav', 'music.wav', 'white-noise.wav', 'tone_10kHz.wav']

audio_speech, sr_speech = sf.read(os.path.join(inp_folder, file_names[0])) # , always_2d=True
audio_street, sr_street = sf.read(os.path.join(inp_folder, file_names[1]))
audio_music, sr_music = sf.read(os.path.join(inp_folder, file_names[2]))
audio_wnoise, sr_wnoise = sf.read(os.path.join(inp_folder, file_names[3]))
audio_test_tone, sr_test_tone = sf.read(os.path.join(inp_folder, file_names[4]))
print("Array Type is: {}".format(audio_music.dtype))
# Attenuate white noise amplitude 
attenuation_ratio = 1/4
audio_files = [audio_speech, audio_street, audio_music, audio_wnoise * attenuation_ratio, audio_test_tone]
sample_rates = [sr_speech, sr_street, sr_music, sr_wnoise, sr_test_tone]

for name, sr, audio in zip(file_names, sample_rates, audio_files):
    print("File: {}, Sample Rate: {}, Samples: {}, Time: {}sec".format(name, sr, len(audio), len(audio)/sr))

In [ ]:
make_mono = True
# Reduce from stereo to mono
if make_mono:
    for ii, audio in enumerate(audio_files):
        if len(audio.shape) == 2:
            peak_l = max(audio[:, 0])
            peak_r = max(audio[:, 1])
            peak_pre = max(peak_l, peak_r)
            audio_files[ii] = np.sum(audio, axis=1)
            audio_files[ii] /= max(audio_files[ii])
            audio_files[ii] *= peak_pre
            print("Shape File {}: {}".format(ii+1, audio_files[ii].shape))       

In [ ]:
import samplerate

max_sr = max(sample_rates)
converter = 'sinc_best'

for ii, (audio, sr) in enumerate(zip(audio_files, sample_rates)):
    if sr != max_sr:
        ratio = max_sr / sr
        audio_files[ii] = samplerate.resample(audio, ratio, converter)
        print("New Shape: {}".format(audio_files[ii].shape))
        sample_rates[ii] = max_sr
        
for name, audio in zip(file_names, audio_files):
    print("File: {}, Sample Rate: {}, Samples: {}, Time: {}sec".format(name, max_sr, len(audio), len(audio)/sr))

In [ ]:
# Find minimal audio length
audio_length = []
for ii, audio in enumerate(audio_files):
    audio_length.append(len(audio))
max_length = min(audio_length)
print("Minimal number of audio samples: {}".format(max_length))

In [ ]:
# Determine Audio length
# Neede samples are samples = time * sample_rate
time = max_length / sample_rates[audio_length.index(max_length)]

adj_audio_length = []
for sr in sample_rates:
    adj_audio_length.append(int(time * sr))

print("Max audio length is {} s".format(time))
for name, smp_has, smp_should in zip(file_names, audio_length, adj_audio_length):
    print("File {} has {} Samples and must have {}".format(name, smp_has, smp_should))

In [ ]:
# Unify audio file length
for ii, smp in enumerate(adj_audio_length):
    audio_files[ii] = audio_files[ii][:smp]
    print("Shape file {}: {}".format(ii+1, audio_files[ii].shape))

# Time-Domain

Time plot of original audio files $u$ and noise sources $v$.

| Audio | Symbol |
| --- | --- |
| Speech | $u_s$ | 
| Street Noise | $v_{sn}$ |
| Music | $u_m$ |
| White Noise | $v_{wn}$ |

In [ ]:
fig, axs = plt.subplots(len(audio_files), 1, figsize=[18, 10], sharex=False, constrained_layout = True)
audio_names = ['Speech $u_s$', 'Street Noise $v_{sn}$', 'Music $u_m$', 'White Noise $v_{wn}$', 'Tone 10kHz']
colors = ['blue', 'red', 'green', 'black', 'orange']
for ii, (audio, name, sr, color) in enumerate(zip(audio_files, audio_names, sample_rates, colors)):
    x_axis = np.arange(0, len(audio)) / sr
    axs[ii].plot(x_axis, audio, lw=5, color=color)
    axs[ii].set_title(name, fontsize=25)

fig.supylabel('Amplitude', fontsize=20)
fig.supxlabel('Time in Sec', fontsize=20)

for ax in axs:
    ax.set_xlim(0)
    ax.tick_params(labelsize=15)
    ax.grid()

# Spectrum

Frequencies in original audio files.

In [ ]:
# Fast Fourier Transform
from scipy.fftpack import fft, fftfreq

spectrums = []
fft_x = []
for audio, sample_rate, samples in zip(audio_files, sample_rates, adj_audio_length):
    frequencies = fft(audio)
    T = 1/sample_rate
    x_axis = fftfreq(samples, T)[:samples//2]
    spectrums.append(2.0/samples * np.abs(frequencies[0:samples//2]))
    fft_x.append(x_axis)


In [ ]:
fig, axs = plt.subplots(len(audio_files), 1, figsize=[18, 10], sharex=False, constrained_layout = True)

for ii, (spectrum, x_axis, name, color) in enumerate(zip(spectrums, fft_x, audio_names, colors)):
    axs[ii].plot(x_axis / 1e3, spectrum, lw=5, color=color)
    axs[ii].set_title(name, fontsize=25)

fig.supylabel('Magnitude', fontsize=20)
fig.supxlabel('Frequency in kHz', fontsize=20)

for ax in axs:
    ax.set_xlim(0)
    ax.tick_params(labelsize=15)
    ax.grid()

In [ ]:
# Save Files
names_source_out = ['speech_source.wav', 'street_source.wav', 'music_source.wav', 'white_noise_source.wav', 'tone_10kHz_source.wav']
for name, audio, sr in zip(names_source_out, audio_files, sample_rates):
    sf.write(os.path.join(outp_folder, name), audio, sr)

# Mix Signals

Generate four observations $d$. 

1. Music + White Noise $d_{m+wn}$
1. Street Noise + Music $d_{sn+m}$
1. Street Noise + Music + Speech $d_{sn+m+s}$
1. Speech + White Noise $d_{s+wn}$

In [ ]:
# 1           Music +       White Noise
d_m_wn = audio_files[2] + audio_files[3]
# 2           Street +      Music
d_sn_m = audio_files[1] + audio_files[2]
# 3 :         Street +      Music +             Speech
d_sn_m_s = audio_files[1] + audio_files[2] + audio_files[0]
# 4 :         Speech +      White Noise
d_s_wn = audio_files[0] + audio_files[3]

observations = [d_m_wn, d_sn_m, d_sn_m_s, d_s_wn]


## Add seed for same results

In [ ]:
np.random.seed(10)

# Independent Component Analysis (ICA)

ICA is a method for seperating a multivariate signal into components. 

Steps:

1. Center $\mathbf{d}$ by subtracting the mean
1. Whiten
1. Choose random initial value for the de-mixing matrix $\mathbf{W}$
1. Calculate the new value for $\mathbf{W}$
1. Normalize $\mathbf{W}$
1. Check whether algorithm has converged and if it hasn’t, return to step 4
1. Take the dot product of $\mathbf{W}$ and $\mathbf{d}$ to get the independent source signals
$$\mathbf{S} = \mathbf{W}\mathbf{d}$$

Source:
* [Independent Component Analysis (ICA) In Python, Cory Maklin](https://towardsdatascience.com/independent-component-analysis-ica-in-python-a0ef0db0955e)
* [Separating mixed signals with Independent Component Analysis, Carsten Klein](https://towardsdatascience.com/separating-mixed-signals-with-independent-component-analysis-38205188f2f4)

In [ ]:
from sklearn.decomposition import FastICA

### Build Observation using True Signals

Generating signals for observation #3 and apply to "cocktail party problem". 

One recording per existing signal is used here. The recorded signal contains a proportion of pollution caused by the other signals.

In [ ]:
# 3 :                   Street +            Music +                 Speech
d_sn_m_s_1 = 0.3 * audio_files[1] + 0.3 * audio_files[2] + 0.4 * audio_files[0]
d_sn_m_s_2 = 0.6 * audio_files[1] + 0.2 * audio_files[2] + 0.2 * audio_files[0]
d_sn_m_s_3 = 0.1 * audio_files[1] + 0.7 * audio_files[2] + 0.2 * audio_files[0]
true_signal = np.c_[audio_files[0], audio_files[1], audio_files[2]]

In [ ]:
fig, axs = plt.subplots(3, 1, figsize=[30, 20], sharex=True, constrained_layout = True)
fig.supylabel('Amplitude', fontsize=20)
fig.supxlabel('Samples', fontsize=20)
fig.suptitle('Street Noise + Music + Speech $d_{sn+m+s}$', fontsize=25)

axs[0].plot(d_sn_m_s_1, lw=5, color='blue')
axs[0].set_title('Signal 1', fontsize=25)
axs[1].plot(d_sn_m_s_2, lw=5, color='red')
axs[1].set_title('Signal 2', fontsize=25)
axs[2].plot(d_sn_m_s_3, lw=5, color='black')
axs[2].set_title('Signal 3', fontsize=25)

for ax in axs: 
    ax.set_xlim(0, max_length)
    ax.tick_params(labelsize=15)
    ax.grid()

### Combine and Normalize Observations

In [ ]:
observation = np.c_[d_sn_m_s_1, d_sn_m_s_2, d_sn_m_s_3]

observation /= observation.std(axis=0)

### Add Additional Distortion

In [ ]:
add_distortion = False
if add_distortion:
    mixing = np.array([[0.5, 1, 0.2],
                        [1, 0.5, 0.4],
                        [0.5, 0.8, 1]])
                        
    observation = np.dot(true_signal, mixing.T) 

In [ ]:
# Save ICA signal
def save_ica_audio(audio, name):
    names_ica = []
    for ii in range(audio.shape[1]):
        names_ica.append('ica_{}_ch{}.wav'.format(name, ii+1))

    for ii, (name, sr) in enumerate(zip(names_ica, sample_rates)):
        sf.write(os.path.join(outp_folder, name), audio[:,ii], sr)

### Apply FastICA

In [ ]:
ica = FastICA() #n_components=3
ica_recovered = ica.fit_transform(observation) 

In [ ]:
simple_plot = False
if simple_plot:
    fig, axs = plt.subplots(3, 1, figsize=[30, 20], sharex=True, constrained_layout = True)
    fig.supylabel('Amplitude', fontsize=20)
    fig.supxlabel('Samples', fontsize=20)
    fig.suptitle('Street Noise + Music + Speech $d_{sn+m+s}$', fontsize=25)
    
    axs[0].plot(ica_recovered[:, 0], lw=5, color='blue')
    axs[0].set_title('ICA Recovered', fontsize=25)
    axs[1].plot(ica_recovered[:, 1], lw=5, color='red')
    axs[2].plot(ica_recovered[:, 2], lw=5, color='black')

    for ax in axs: 
        ax.set_xlim(0, max_length)
        ax.tick_params(labelsize=15)
        ax.grid()
else:
    ica_lst = [observation, true_signal, ica_recovered]
    ica_name = ['Observation', 'True Signal', 'ICA Recovered']
    fig, axs = plt.subplots(3, 3, figsize=[30, 20], sharex=True, constrained_layout = True)
    fig.supylabel('Amplitude', fontsize=20)
    fig.supxlabel('Samples', fontsize=20)
    fig.suptitle('Street Noise + Music + Speech $d_{sn+m+s}$', fontsize=25)

    for ii, (data, name) in enumerate(zip(ica_lst, ica_name)):
        axs[0][ii].plot(data[:, 0], lw=5, color='blue')
        axs[0][ii].set_title(name, fontsize=25)
        axs[1][ii].plot(data[:, 1], lw=5, color='red')
        axs[2][ii].plot(data[:, 2], lw=5, color='black')

    for row in axs: 
        for ax in row: 
            ax.set_xlim(0, max_length)
            ax.tick_params(labelsize=15)
            ax.grid()

In [ ]:
# Save observations and recovered signal
save_ica_audio(ica_recovered, 'recv_sig3')
save_ica_audio(observation, 'obs_sig3')

# Perform ICA for the remaining test observations

In [ ]:
def ica(true_sig, observation, signal_name, add_distort=False):
    true_sig /= true_sig.std(axis=0)
    observation /= observation.std(axis=0)

    if add_distort:
        mixing = np.array([[0.5, 1],
                            [0.8, 0.2]])
        observation = np.dot(true_sig, mixing.T)
        
    ica = FastICA()
    ica_recovered = ica.fit_transform(observation)

    ica_lst = [observation, true_sig, ica_recovered]
    ica_name = ['Observation', 'True Signal', 'ICA Recovered']
    fig, axs = plt.subplots(2, 3, figsize=[30, 20], sharex=True, constrained_layout = True)
    fig.supylabel('Amplitude', fontsize=20)
    fig.supxlabel('Samples', fontsize=20)

    save_ica_audio(ica_recovered, 'recv_{}'.format(signal_name))
    save_ica_audio(observation, 'obs_{}'.format(signal_name))

    equalize_amp = False
    if equalize_amp:
        ratio = np.abs(true_sig).mean() / np.abs(ica_recovered).mean()
        for ii in range(ica_recovered.shape[1]):
            ica_recovered[:, ii] = ica_recovered[:, ii] * ratio
    
    for ii, (data, name) in enumerate(zip(ica_lst, ica_name)):
        axs[0][ii].plot(data[:, 0], lw=5, color='blue')
        axs[0][ii].set_title(name, fontsize=25)
        axs[1][ii].plot(data[:, 1], lw=5, color='red')

    for row in axs: 
        for ax in row: 
            ax.set_xlim(0, max_length)
            ax.tick_params(labelsize=15)
            ax.grid()



## Remaining Signals

In [ ]:
# 1                 Music +             White Noise
d_m_wn_1 = 0.7 * audio_files[2] + 0.3 * audio_files[3]
d_m_wn_2 = 0.3 * audio_files[2] + 0.7 * audio_files[3]
observation_1 = np.c_[d_m_wn_1, d_m_wn_2]
true_signal_1 = np.c_[audio_files[2], audio_files[3]]
# 2                 Street +            Music
d_sn_m_1 = 0.7 * audio_files[1] + 0.3 * audio_files[2]
d_sn_m_2 = 0.3 * audio_files[1] + 0.7 * audio_files[2]
observation_2 = np.c_[d_sn_m_1, d_sn_m_2]
true_signal_2 = np.c_[audio_files[1], audio_files[2]]
# 4 :               Speech +            White Noise
d_s_wn_1 = 0.7 * audio_files[0] + 0.3 * audio_files[3]
d_s_wn_2 = 0.3 * audio_files[0] + 0.7 * audio_files[3]
observation_4 = np.c_[d_s_wn_1, d_s_wn_2]
true_signal_4 = np.c_[audio_files[0], audio_files[3]]

# Music + White Noise $d_{m+wn}$

In [ ]:
ica(true_signal_1, observation_1, 'sig1')

# Street Noise + Music $d_{sn+m}$

In [ ]:
ica(true_signal_2, observation_2, 'sig2')

# Speech + White Noise $d_{s+wn}$

In [ ]:
ica(true_signal_4, observation_4, 'sig4')

# Wavelet Transform

Source:
* [Audio Classification using Wavelet Transform and Deep Learning, Aditya Dutt](https://medium.com/mlearning-ai/audio-classification-using-wavelet-transform-and-deep-learning-f9f0978fa246)
* [Choose a Wavelet, MathWorks](https://de.mathworks.com/help/wavelet/gs/choose-a-wavelet.html)

In [ ]:
import pywt

In [ ]:
# print the wavelet families available
print(pywt.families())
# print a list of available wavelets from one family
print(pywt.wavelist(kind='discrete'))

In [ ]:
wavelet = 'db5'
mode = 'symmetric'

In [ ]:
def calc_wavelet(signal, wavelet, mode):
    coeffs = pywt.wavedec(signal, wavelet, mode=mode)
    return coeffs

In [ ]:
def plot_wavelet(coeffs, signal):
    fig, axs = plt.subplots(len(coeffs)+1, 1, figsize=[18, 30], sharex=True, constrained_layout = True)
    fig.supylabel('Amplitude', fontsize=20)

    axs[0].plot(signal, lw=5)
    axs[0].set_title('Mixed Data', fontsize=25)

    for ii, coeff in enumerate(coeffs):
        axs[ii+1].plot(coeff, lw=5)
        axs[ii+1].set_title('c'+ str(ii), fontsize=25)

    axs[len(coeffs)].set_xlabel('Samples', fontsize=20)

    for ax in axs:
        ax.set_xlim(0)
        ax.tick_params(labelsize=15)
        ax.grid()

In [ ]:
def recover_wavelet(coeffs, remove_st, remove_end):
    ctn = list(range(remove_st, remove_end+1))
    for ii in ctn:
        coeffs[-ii] = np.zeros_like(coeffs[-ii])

    return pywt.waverec(coeffs, wavelet, mode=mode)

# Speech + White Noise $d_{s+wn}$

In [ ]:
coeff_dswn = calc_wavelet(d_s_wn, wavelet, mode)
plot_wavelet(coeff_dswn, d_s_wn)

In [ ]:
recov_dswn = recover_wavelet(coeff_dswn, 0, 5)

fig, axs = plt.subplots(3, 1, figsize=[18, 10], sharex=True, constrained_layout = True)
fig.supylabel('Amplitude', fontsize=20)

axs[0].plot(d_s_wn, lw=5)
axs[0].set_title('Mixed Data', fontsize=25)

axs[1].plot(recov_dswn, lw=5)
axs[1].set_title('Recovered Signal', fontsize=25)

axs[2].plot(audio_files[0], lw=5)
axs[2].set_title('Speech Signal', fontsize=25)

for ax in axs:
    ax.set_xlim(0)
    ax.tick_params(labelsize=15)
    ax.grid()

# Street Noise + Music $d_{sn+m}$

In [ ]:
coeff_dsnm = calc_wavelet(d_sn_m, wavelet, mode)

In [ ]:
plot_wavelet(coeff_dsnm, d_sn_m)

In [ ]:
recov_dsnm = recover_wavelet(coeff_dsnm, 0, 5)

fig, axs = plt.subplots(4, 1, figsize=[18, 10], sharex=True, constrained_layout = True)
fig.supylabel('Amplitude', fontsize=20)

axs[0].plot(d_sn_m, lw=5)
axs[0].set_title('Mixed Data', fontsize=25)

axs[1].plot(recov_dsnm, lw=5)
axs[1].set_title('Recovered Signal', fontsize=25)

axs[2].plot(audio_files[1], lw=5)
axs[2].set_title('Street Noise', fontsize=25)

axs[3].plot(audio_files[2], lw=5)
axs[3].set_title('Music Signal', fontsize=25)

for ax in axs:
    ax.set_xlim(0)
    ax.tick_params(labelsize=15)
    ax.grid()

# Music + White Noise $d_{m+wn}$

In [ ]:
coeff_dmwn = calc_wavelet(d_m_wn, wavelet, mode)

In [ ]:
plot_wavelet(coeff_dmwn, d_m_wn)

In [ ]:
recov_dmwn = recover_wavelet(coeff_dmwn, 0, 4)

fig, axs = plt.subplots(4, 1, figsize=[18, 10], sharex=True, constrained_layout = True)
fig.supylabel('Amplitude', fontsize=20)

axs[0].plot(d_m_wn, lw=5)
axs[0].set_title('Mixed Data', fontsize=25)

axs[1].plot(recov_dmwn, lw=5)
axs[1].set_title('Recovered Signal', fontsize=25)

axs[2].plot(audio_files[3], lw=5)
axs[2].set_title('White Noise', fontsize=25)

axs[3].plot(audio_files[2], lw=5)
axs[3].set_title('Music Signal', fontsize=25)

for ax in axs:
    ax.set_xlim(0)
    ax.tick_params(labelsize=15)
    ax.grid()

# Street Noise + Music + Speec $d_{sn+m+s}$

In [ ]:
coeff_snms = calc_wavelet(d_sn_m_s, wavelet, mode)

In [ ]:
plot_wavelet(coeff_snms, d_sn_m_s)

In [ ]:
recov_snms = recover_wavelet(coeff_snms, 0, 5)

fig, axs = plt.subplots(5, 1, figsize=[18, 10], sharex=True, constrained_layout = True)
fig.supylabel('Amplitude', fontsize=20)

axs[0].plot(d_sn_m_s, lw=5)
axs[0].set_title('Mixed Data', fontsize=25)

axs[1].plot(recov_snms, lw=5)
axs[1].set_title('Recovered Signal', fontsize=25)

axs[2].plot(audio_files[1], lw=5)
axs[2].set_title('Street Noise', fontsize=25)

axs[3].plot(audio_files[2], lw=5)
axs[3].set_title('Music Signal', fontsize=25)

axs[4].plot(audio_files[0], lw=5)
axs[4].set_title('Speech Signal', fontsize=25)

for ax in axs:
    ax.set_xlim(0)
    ax.tick_params(labelsize=15)
    ax.grid()

# Short Time Fourier Transform (STFT)

In [ ]:
import scipy.signal as signal

# Street Noise + Music + Speec $d_{sn+m+s}$

In [ ]:
f, t, Zxx = signal.stft(d_sn_m_s, fs=sample_rates[0], nperseg=1000)
plt.figure(figsize=(10,5))
plt.pcolormesh(t, f, np.abs(Zxx))
plt.title('STFT Magnitude')
plt.ylabel('Frequency [Hz]')
plt.xlabel('Time [sec]')
plt.ylim(0, 7000)

# Music + White Noise $d_{m+wn}$

In [ ]:
f, t, Zxx = signal.stft(d_m_wn, fs=sample_rates[0], nperseg=1000)
plt.figure(figsize=(10,5))
plt.pcolormesh(t, f, np.abs(Zxx))
plt.title('STFT Magnitude')
plt.ylabel('Frequency [Hz]')
plt.xlabel('Time [sec]')
plt.ylim(0, 7000)

# Speech + White Noise $d_{s+wn}$

In [ ]:
f, t, Zxx = signal.stft(d_s_wn, fs=sample_rates[0], nperseg=1000)
plt.figure(figsize=(10,5))
plt.pcolormesh(t, f, np.abs(Zxx))
plt.title('STFT Magnitude')
plt.ylabel('Frequency [Hz]')
plt.xlabel('Time [sec]')
plt.ylim(0, 7000)

# Street Noise + Music $d_{sn+m}$

In [ ]:
f, t, Zxx = signal.stft(d_sn_m, fs=sample_rates[0], nperseg=1000)
plt.figure(figsize=(10,5))
plt.pcolormesh(t, f, np.abs(Zxx))
plt.title('STFT Magnitude')
plt.ylabel('Frequency [Hz]')
plt.xlabel('Time [sec]')
plt.ylim(0, 7000)

# Adaptive Filter

In [ ]:
import padasip as pa

# Speech + White Noise $d_{s+wn}$

In [ ]:
samples = len(audio_files[0])
memory = 10
u = audio_files[0]
v = audio_files[3]
d = d_s_wn

x = pa.input_from_history(d, memory)[:-1]
d = d[memory:]
u = u[memory:]
f = pa.filters.FilterRLS(mu=.5, n=memory)
y, e, w = f.run(d, x)

plt.figure(figsize=(12.5,6))
plt.plot(u, "r:", linewidth=4, label="original")
plt.plot(d, "b", label="noisy")
plt.plot(y, "g", label="filtered")
plt.xlim(60000, 60100)
plt.legend()
plt.ylabel('Amplitude')
plt.xlabel('Samples')
plt.tight_layout()
plt.grid()